In [2]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse import identity
from scipy.sparse import kron
from tqdm import tqdm

from qiskit.quantum_info import Pauli

In [3]:
def build_operators(L: int, N: int) -> list:
    """Generate all the annihilation operators needed to build the hamiltonian

    Args:
        L (int): the cutoff of the single site Fock space
        N (int): the number of colors of gauge group SU(N)

    Returns:
        list: a list of annihilation operators, length=N_bos
    """
    # The annihilation operator for the single boson
    a_b = diags(np.sqrt(np.linspace(1, L - 1, L - 1)), offsets=1)
    # The identity operator of the Fock space of a single boson
    i_b = identity(L)
    # Bosonic Hilbert space
    N_bos = int(2 * (N ** 2 - 1))  # number of boson sites -> fixed for mini-BMN 2
    product_list = [i_b] * N_bos  # only the identity for bosons repeated N_bos times
    a_b_list = []  # this will contain a1...a6
    for i in np.arange(0, N_bos):  # loop over all bosonic operators
        operator_list = product_list.copy()  # all elements are the identity operator
        operator_list[
            i
        ] = a_b  # the i^th element is now the annihilation operator for a single boson
        a_b_list.append(
            operator_list[0]
        )  # start taking tensor products from first element
        for a in operator_list[1:]:
            a_b_list[i] = kron(
                a_b_list[i], a
            )  # do the outer product between each operator_list element
    return a_b_list


# %%
def build_gauge_casimir(L: int, N: int):
    """Generate the gauge generators operators

    Args:
        L (int): the single site cutoff of the Fock space
        N (int): the number of colors in the gauge group SU(N)

    Returns:
        scipy.sparse : The sparse matrix for \sum_i G_i^2
    """
    # generate the annihilation operators
    bosons = build_operators(L, N)
    # define the generator list for SU(2)
    g_list = [0] * 3
    g_list[0] = 1j * (
        bosons[1].conjugate().transpose() * bosons[2]
        - bosons[2].conjugate().transpose() * bosons[1]
        + bosons[4].conjugate().transpose() * bosons[5]
        - bosons[5].conjugate().transpose() * bosons[4]
    )
    g_list[1] = 1j * (
        bosons[2].conjugate().transpose() * bosons[0]
        - bosons[0].conjugate().transpose() * bosons[2]
        + bosons[5].conjugate().transpose() * bosons[3]
        - bosons[3].conjugate().transpose() * bosons[5]
    )
    g_list[2] = 1j * (
        bosons[0].conjugate().transpose() * bosons[1]
        - bosons[1].conjugate().transpose() * bosons[0]
        + bosons[3].conjugate().transpose() * bosons[4]
        - bosons[4].conjugate().transpose() * bosons[3]
    )

    return g_list[0] * g_list[0] + g_list[1] * g_list[1] + g_list[2] * g_list[2]


# %%
def bmn2_hamiltonian(L: int = 2, N: int = 2, g2N: float = 0.2):
    """Construct the Hamiltonian of the bosonic BMN model as a sparse matrix.
    The cutoff for each boson is L while the 't Hooft coupling in g2N for a gauge group SU(N).
    The limited number of qubits only let us simulate N=2 and L=4 => for 6 bosons this is a 12 qubits problem.

    Args:
        L (int, optional): The cutoff of the bosonic modes (the annihilation operators will be LxL matrices). Defaults to 2.
        N (int, optional): The number of colors of a SU(N) gauge group. The degrees of freedom of one matrix will be N^2-1. Defaults to 2.
        g2N (float, optional): The 't Hooft coupling. Defaults to 0.2.
    """
    print(
        f"Building bosonic BMN Hamiltonian for SU({N}) with cutoff={L} and coupling={g2N}\n"
    )
    a_b_list = build_operators(L,N)
    N_bos = int(2 * (N ** 2 - 1))  # number of boson sites -> FIXED for mini-BMN 2
    # Build the Hamiltonian
    # Start piece by piece
    x_list = []
    # only use the bosonic operators
    for op in a_b_list:
        x_list.append(1 / np.sqrt(2) * (op.conjugate().transpose() + op))
    # Free Hamiltonian
    H_k = 0
    for a in a_b_list:
        H_k = H_k + a.conjugate().transpose() * a
    # vacuum energy
    H_k = H_k + 0.5 * N_bos * identity(L ** N_bos)
    # Interaction among bosons
    V_b = (
        x_list[2] * x_list[2] * x_list[3] * x_list[3]
        + x_list[2] * x_list[2] * x_list[4] * x_list[4]
        + x_list[1] * x_list[1] * x_list[3] * x_list[3]
        + x_list[1] * x_list[1] * x_list[5] * x_list[5]
        + x_list[0] * x_list[0] * x_list[4] * x_list[4]
        + x_list[0] * x_list[0] * x_list[5] * x_list[5]
        - 2 * x_list[0] * x_list[2] * x_list[3] * x_list[5]
        - 2 * x_list[0] * x_list[1] * x_list[3] * x_list[4]
        - 2 * x_list[1] * x_list[2] * x_list[4] * x_list[5]
    )
    # full hamiltonian
    return H_k + g2N / N * V_b

In [4]:
mat = bmn2_hamiltonian(L=4, N=2)

Building bosonic BMN Hamiltonian for SU(2) with cutoff=4 and coupling=0.2



In [5]:
np.trace(mat @ Pauli('IIIIIIIIIIII'))

(54681.6+0j)

In [6]:
import pickle
with open("./H_12q.pk", "rb") as f: 
    paulis, coeffs = pickle.load(f)

In [7]:
paulis = [Pauli(x) for x in paulis]

In [8]:
tmp = 0
for p in tqdm(paulis):
    tmp += 1/2**12 * p.to_matrix() * np.trace(mat @ p)  

  1%|          | 10/889 [00:08<12:30,  1.17it/s]


KeyboardInterrupt: 

In [25]:
tmp.shape

(4096, 4096)

In [26]:
diff = mat.toarray() - tmp
diff

array([[-3.00000000e+00+0.j,  0.00000000e+00+0.j,  1.38777878e-17+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j, -2.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       [ 1.38777878e-17+0.j,  0.00000000e+00+0.j, -3.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       ...,
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  3.00000000e+00+0.j,  0.00000000e+00+0.j,
         1.66533454e-16+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  2.00000000e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  1.66533454e-16+0.j,  0.00000000e+00+0.j,
         3.00000000e+00+0.j]])

In [28]:
out = []
for i in tqdm(range(diff.shape[0])):
    for j in range(diff.shape[0]):
        if np.isclose(diff[i, j], 0):
            diff[i, j] = 0
        else:
            out.append((i, j, diff[i, j]))
out            

100%|██████████| 4096/4096 [02:19<00:00, 29.32it/s]


[(0, 0, (-3.000000000000004+0j)),
 (1, 1, (-2.0000000000000018+0j)),
 (2, 2, (-3+0j)),
 (3, 3, (-2.0000000000000018+0j)),
 (4, 4, (-2.0000000000000018+0j)),
 (5, 5, (-1+0j)),
 (6, 6, (-1.9999999999999973+0j)),
 (7, 7, (-1+0j)),
 (8, 8, (-3+0j)),
 (9, 9, (-1.9999999999999973+0j)),
 (10, 10, (-2.9999999999999947+0j)),
 (11, 11, (-1.9999999999999982+0j)),
 (12, 12, (-2.0000000000000018+0j)),
 (13, 13, (-1+0j)),
 (14, 14, (-1.9999999999999982+0j)),
 (15, 15, (-1+0j)),
 (16, 16, (-2.0000000000000018+0j)),
 (17, 17, (-1+0j)),
 (18, 18, (-1.9999999999999973+0j)),
 (19, 19, (-1+0j)),
 (20, 20, (-1+0j)),
 (22, 22, (-0.9999999999999956+0j)),
 (24, 24, (-1.9999999999999973+0j)),
 (25, 25, (-0.9999999999999956+0j)),
 (26, 26, (-1.999999999999993+0j)),
 (27, 27, (-0.9999999999999947+0j)),
 (28, 28, (-1+0j)),
 (30, 30, (-0.9999999999999947+0j)),
 (32, 32, (-3+0j)),
 (33, 33, (-1.9999999999999973+0j)),
 (34, 34, (-2.9999999999999947+0j)),
 (35, 35, (-1.9999999999999982+0j)),
 (36, 36, (-1.99999999999

In [30]:
for i, j, val in out:
    if i != j:
        print((i, j, val))

In [9]:
import pickle
with open("./H_12q.pk", "rb") as f: 
    paulis, coeffs = pickle.load(f)

In [10]:
import itertools

def generate_all_strings(size=12, existing_paulis=None):
    letters = ['Z', 'I']
    excluded = set(existing_paulis) if existing_paulis else set()
    
    result = []
    for p in tqdm(itertools.product(letters, repeat=size)):
        candidate = ''.join(p)
        if candidate not in excluded:
            result.append(candidate)
    
    return result

all_other_z_paulis = generate_all_strings(existing_paulis=paulis)
all_other_z_paulis

4096it [00:00, 1370224.05it/s]


['ZZZZZZZZZZZZ',
 'ZZZZZZZZZZZI',
 'ZZZZZZZZZZIZ',
 'ZZZZZZZZZZII',
 'ZZZZZZZZZIZZ',
 'ZZZZZZZZZIZI',
 'ZZZZZZZZZIIZ',
 'ZZZZZZZZZIII',
 'ZZZZZZZZIZZZ',
 'ZZZZZZZZIZZI',
 'ZZZZZZZZIZIZ',
 'ZZZZZZZZIZII',
 'ZZZZZZZZIIZZ',
 'ZZZZZZZZIIZI',
 'ZZZZZZZZIIIZ',
 'ZZZZZZZZIIII',
 'ZZZZZZZIZZZZ',
 'ZZZZZZZIZZZI',
 'ZZZZZZZIZZIZ',
 'ZZZZZZZIZZII',
 'ZZZZZZZIZIZZ',
 'ZZZZZZZIZIZI',
 'ZZZZZZZIZIIZ',
 'ZZZZZZZIZIII',
 'ZZZZZZZIIZZZ',
 'ZZZZZZZIIZZI',
 'ZZZZZZZIIZIZ',
 'ZZZZZZZIIZII',
 'ZZZZZZZIIIZZ',
 'ZZZZZZZIIIZI',
 'ZZZZZZZIIIIZ',
 'ZZZZZZZIIIII',
 'ZZZZZZIZZZZZ',
 'ZZZZZZIZZZZI',
 'ZZZZZZIZZZIZ',
 'ZZZZZZIZZZII',
 'ZZZZZZIZZIZZ',
 'ZZZZZZIZZIZI',
 'ZZZZZZIZZIIZ',
 'ZZZZZZIZZIII',
 'ZZZZZZIZIZZZ',
 'ZZZZZZIZIZZI',
 'ZZZZZZIZIZIZ',
 'ZZZZZZIZIZII',
 'ZZZZZZIZIIZZ',
 'ZZZZZZIZIIZI',
 'ZZZZZZIZIIIZ',
 'ZZZZZZIZIIII',
 'ZZZZZZIIZZZZ',
 'ZZZZZZIIZZZI',
 'ZZZZZZIIZZIZ',
 'ZZZZZZIIZZII',
 'ZZZZZZIIZIZZ',
 'ZZZZZZIIZIZI',
 'ZZZZZZIIZIIZ',
 'ZZZZZZIIZIII',
 'ZZZZZZIIIZZZ',
 'ZZZZZZIIIZZI',
 'ZZZZZZIIIZIZ

In [11]:
print(len(all_other_z_paulis))

4059


In [ ]:
for p in tqdm(all_other_z_paulis):
    if not np.isclose(np.trace(mat @ Pauli(p)), 0):
        print(p, np.real_if_close(np.trace(mat @ Pauli(p))))

In [13]:
def worker(p):
    """
    Process a single Pauli string 'p'. Compute the trace of (mat @ Pauli(p))
    and return (p, trace_value) if the trace is not close to zero.
    Otherwise, return None.
    """
    # Construct the Pauli matrix from the string.
    pauli_matrix = Pauli(p)
    trace_val = np.trace(mat @ pauli_matrix)
    
    if not np.isclose(trace_val, 0):
        # np.real_if_close returns a real number if the result is close to real.
        return (p, np.real_if_close(trace_val))
    return None

In [14]:
import multiprocessing as mp
with mp.Pool() as pool:
    # Use imap_unordered for parallel mapping; tqdm displays a progress bar.
    for result in tqdm(pool.imap_unordered(worker, all_other_z_paulis),
                        total=len(all_other_z_paulis)):
        if result is not None:
            p_string, trace_value = result
            print(p_string, trace_value)

 76%|███████▌  | 3079/4059 [00:43<00:13, 74.73it/s] 

IZIIIIIIIIII -2047.9999999999982


 94%|█████████▍| 3822/4059 [00:53<00:03, 78.36it/s]

IIIZIIIIIIII -2048.000000000001


 99%|█████████▉| 4026/4059 [00:55<00:00, 118.70it/s]

IIIIIZIIIIII -2047.999999999999
IIIIIIIIIZII -2047.9999999999995


100%|██████████| 4059/4059 [00:56<00:00, 72.39it/s] 


IIIIIIIZIIII -2047.9999999999995
IIIIIIIIIIIZ -2048.0
